In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../../")

from autogluon.vision import ImagePredictor, ImageDataset
import numpy as np
import pandas as pd
import pickle
import datetime
from pathlib import Path
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from IPython.display import HTML
from matplotlib import pyplot as plt
from cleanlab.internal.label_quality_utils import get_normalized_entropy

from approximate_nearest_neighbors import ApproxNearestNeighbors

# pre-trained model
from image_feature_extraction.extract_features_from_image_dir import extract_features_from_image_dir

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Read data

In [2]:
model = "swin_base_patch4_window7_224" 

data_model_dict = {
    "cifar-10": {"data_path": "/Data/cifar10_png/"},
    "cifar-100": {"data_path": "/Data/cifar100_png/"},
    "roman-numeral": {"data_path": "/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/"},
    "mnist": {"data_path": "/Data/mnist_png/mnist_png/"},
    "fashion-mnist": {"data_path": "/Data/fashion_mnist_png/"}
}

# path to pre-trained model in ONNX format
path_to_onnx = "../../image_feature_extraction/models/feature_extractor.onnx"

# Get data, model, and pre-trained features
for dataset in data_model_dict.keys():
    
    print("--------------------------")
    print(f"Getting data for {dataset}")
    
    # Get path to data
    data_path = data_model_dict[dataset]["data_path"]
    
    # Get train and test data
    data_model_dict[dataset]["train_data"], _, data_model_dict[dataset]["test_data"] = \
        ImageDataset.from_folders(root=data_path)
    
    # Get path to saved model
    data_model_dict[dataset]["model"] = f"./autogluon_models/{model}_{dataset}.ag"
    
    # Get pre-trained features; ResNet50 model pre-trained on ImageNet
    # NOTE: we will compare these with LEARNED embeddings from the trained model (see below)
    print(f"Extracting pre-trained embeddings...")
    for split in ["train", "test"]:
        data_model_dict[dataset][f"{split}_data_pretrained_features"], _, _ = \
            extract_features_from_image_dir(path_to_onnx=path_to_onnx, path_to_image_dir=f"{data_path}{split}/", batch_size=128)

--------------------------
Getting data for cifar-10
Extracting pre-trained embeddings...
Function: init_data_loader_from_image_folder, Time: 0.3103651300043566 Seconds
Function: read_images_as_data_loader, Time: 0.3106159250019118 Seconds
Starting ONNX runtime engine...
  ONNX runtime device: GPU
  ONNX runtime session providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Extracting features...
Function: extract_features_from_image_dir, Time: 33.73509187999298 Seconds
Function: init_data_loader_from_image_folder, Time: 0.06405507599993143 Seconds
Function: read_images_as_data_loader, Time: 0.06429411900171544 Seconds
Starting ONNX runtime engine...
  ONNX runtime device: GPU
  ONNX runtime session providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Extracting features...
Function: extract_features_from_image_dir, Time: 5.223168789001647 Seconds
--------------------------
Getting data for cifar-100
Extracting pre-trained embeddings...
Function: init_data_loader_from_ima

## Evaluate models on test data as a sanity check

In [3]:
accuracy_result_list = []

for key, val in data_model_dict.items():
    
    dataset = key
    model_path = val["model"]
    test_dataset = val["test_data"]
    
    print(f"Dataset: {dataset}")
    
    # load model
    print("  Loading model...")
    predictor_loaded = ImagePredictor.load(model_path)
    
    # evaluating model on test data
    print("  Evaluating model...")
    eval_ = predictor_loaded.evaluate(test_dataset)
    print(f"    Evaluation: {eval_}")
    
    accuracy_result = {
        "dataset": dataset,
        "top1": eval_["top1"]
    }
    
    accuracy_result_list.append(accuracy_result)

Dataset: cifar-10
  Loading model...


/usr/local/lib/python3.8/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  Evaluating model...


[Epoch 24] validation: top1=0.988700 top5=0.999900


    Evaluation: {'loss': 0.11967918761968613, 'top1': 0.9887, 'top5': 0.9999}
Dataset: cifar-100
  Loading model...
  Evaluating model...


...

    Evaluation: {'loss': 0.6779465489643665, 'top1': 0.7966942148760331, 'top5': 0.9805785123966942}
Dataset: mnist
  Loading model...
  Evaluating model...


[Epoch 24] validation: top1=0.991400 top5=1.000000


    Evaluation: {'loss': 0.11047064321041107, 'top1': 0.9914, 'top5': 1.0}
Dataset: fashion-mnist
  Loading model...
  Evaluating model...


[Epoch 37] validation: top1=0.948900 top5=0.999700


    Evaluation: {'loss': 0.2166707287788391, 'top1': 0.9489, 'top5': 0.9997}


## Evaluate OOD Scores on TEST data

In [30]:
%%time

# pairs of (in-distribution, out-of-distribution) datasets to evaluate
in_out_pairs = [
    {"in": "cifar-10", "out": "cifar-100"},
    {"in": "cifar-100", "out": "cifar-10"},
    {"in": "mnist", "out": "roman-numeral"},
    {"in": "roman-numeral", "out": "mnist"},
    {"in": "mnist", "out": "fashion-mnist"},
    {"in": "fashion-mnist", "out": "mnist"},
]

k_max = 110 # max k value for K nearest neighbor search

results_list = []

for in_out_pair in in_out_pairs:
    
    in_dataset, out_dataset = in_out_pair["in"], in_out_pair["out"]
    
    # path to model trained on in-distribution train dataset
    in_model_path = data_model_dict[in_dataset]["model"]

    # get TEST datasets used for evaluation
    in_test_dataset = data_model_dict[in_dataset]["test_data"]
    out_test_dataset = data_model_dict[out_dataset]["test_data"]
    
    # class labels for the in-distribution test dataset
    in_test_dataset_class_labels = in_test_dataset.label.values    
    
    print("-----------------------------------------------------")
    print("(in-distribution, out-of-distribution) dataset pair: ", in_dataset, out_dataset)
    
    # load model
    print("  Loading model...")
    in_predictor_loaded = ImagePredictor.load(in_model_path)
    
    # Get test predicted probabilities
    print("  Generating predicted probabilities...")
    in_pred_probs = in_predictor_loaded.predict_proba(data=in_test_dataset, as_pandas=False)
    out_pred_probs = in_predictor_loaded.predict_proba(data=out_test_dataset, as_pandas=False)
    
    # Get LEARNED embeddings
    print("  Extracting learned embeddings...")
    in_features = \
        np.stack(
            in_predictor_loaded.predict_feature(data=in_test_dataset, as_pandas=False)[:, 0]
        )
    
    out_features = \
        np.stack(
            in_predictor_loaded.predict_feature(data=out_test_dataset, as_pandas=False)[:, 0]
        )
    
    # Get pre-trained embeddings
    in_pretrained_features = data_model_dict[in_dataset]["test_data_pretrained_features"]
    out_pretrained_features = data_model_dict[out_dataset]["test_data_pretrained_features"]
    
    # Combine pred_probs and features
    pred_probs = np.vstack([in_pred_probs, out_pred_probs])
    features = np.vstack([in_features, out_features]) # LEARNED embeddings
    pretrained_features = np.vstack([in_pretrained_features, out_pretrained_features])

    # Create binary labels (1 = out-of-distribution)
    in_labels = np.zeros(shape=len(in_pred_probs))
    out_labels = np.ones(shape=len(out_pred_probs))
    labels = np.hstack([in_labels, out_labels]) # OOD binary indicator
    
    print("  Running nearest neighbors search...")
    #### Compute nearest neighbors
    
    # nearest neighbors
    nns = ApproxNearestNeighbors(
            features=features, # LEARNED embeddings from the trained model
            labels=labels,
            ) # init Nearest Neighbor Scorer
    nns.build_index() # build index for nearest neighbor lookup
    neighbors_idx, neighbors_dist, neighbors_labels = nns.get_k_nearest_neighbors(k=k_max)
    
    print("  Generating scores...")
    #### Generate scores
    one_minus_max_pred_prob = 1. - pred_probs.max(axis=1)
    
    entropy = get_normalized_entropy(pred_probs)
    get_neighbor_entropy = np.vectorize(lambda idx: entropy[idx]) # Used to get entropy of neighbors
    
    get_neighbor_pred_probs = np.vectorize(lambda idx: pred_probs[idx], signature='()->(n)') # Used to get pred_probs of neighbors
    
    knn_k1_dist = neighbors_dist[:, :1].mean(axis=1)
    
    k = 5
    knn_k5_entropy = (get_neighbor_entropy(neighbors_idx[:,:k]).sum(axis=1) + entropy) / (k + 1)
    knn_k5_dist = neighbors_dist[:, :k].mean(axis=1)    
    
    k = 10
    knn_k10_entropy = (get_neighbor_entropy(neighbors_idx[:,:k]).sum(axis=1) + entropy) / (k + 1)
    knn_k10_dist = neighbors_dist[:, :k].mean(axis=1)
    
    k = 15
    knn_k15_entropy = (get_neighbor_entropy(neighbors_idx[:,:k]).sum(axis=1) + entropy) / (k + 1)    
    knn_k15_dist = neighbors_dist[:, :k].mean(axis=1)
    
    k = 100
    knn_k100_entropy = (get_neighbor_entropy(neighbors_idx[:,:k]).sum(axis=1) + entropy) / (k + 1)    
    knn_k100_dist = neighbors_dist[:, :k].mean(axis=1)
    
    # anomaly score (isolation forest)
    anomaly_model = IsolationForest(random_state=0, n_estimators=100) # instantiate model
    anomaly_model.fit(features) # out-of-sample extracted features
    anomaly_score = 1 / anomaly_model.score_samples(features) # take the inverse so higher scores are more anomalous
    
    # entropy of KNN avg pred_probs
    k = 10
    neighbors_pred_probs = (get_neighbor_pred_probs(neighbors_idx[:,:k]).sum(axis=1) + pred_probs) / (k + 1)
    entropy_knn_k10_pred_probs = get_normalized_entropy(neighbors_pred_probs)
    
        
    #### Evaluate scores
    auroc_one_minus_max_pred_prob = roc_auc_score(labels, one_minus_max_pred_prob)
    
    auroc_entropy = roc_auc_score(labels, entropy)

    auroc_knn_k5_entropy = roc_auc_score(labels, knn_k5_entropy)
    auroc_knn_k10_entropy = roc_auc_score(labels, knn_k10_entropy)
    auroc_knn_k15_entropy = roc_auc_score(labels, knn_k15_entropy)
    auroc_knn_k100_entropy = roc_auc_score(labels, knn_k100_entropy)
    
    auroc_entropy_knn_k10_pred_probs = roc_auc_score(labels, entropy_knn_k10_pred_probs)        

    auroc_knn_k1_dist = roc_auc_score(labels, knn_k1_dist)
    auroc_knn_k5_dist = roc_auc_score(labels, knn_k5_dist)
    auroc_knn_k10_dist = roc_auc_score(labels, knn_k10_dist)
    auroc_knn_k15_dist = roc_auc_score(labels, knn_k15_dist)    
    auroc_knn_k100_dist = roc_auc_score(labels, knn_k100_dist)        

    auroc_anomaly_score = roc_auc_score(labels, anomaly_score)
    
    results = {
        "in_distribution": in_dataset,
        "out_of_distribution": out_dataset,
        "auroc_one_minus_max_pred_prob": auroc_one_minus_max_pred_prob,
        "auroc_entropy": auroc_entropy,
        
        "auroc_knn_k5_entropy": auroc_knn_k5_entropy,
        "auroc_knn_k10_entropy": auroc_knn_k10_entropy,
        "auroc_knn_k15_entropy": auroc_knn_k15_entropy,
        "auroc_knn_k100_entropy": auroc_knn_k100_entropy,
        
        "auroc_entropy_knn_k10_pred_probs": auroc_entropy_knn_k10_pred_probs,

        "auroc_knn_k1_dist": auroc_knn_k1_dist,
        "auroc_knn_k5_dist": auroc_knn_k5_dist,
        "auroc_knn_k10_dist": auroc_knn_k10_dist,
        "auroc_knn_k15_dist": auroc_knn_k15_dist,
        "auroc_knn_k100_dist": auroc_knn_k100_dist,
        
        "auroc_anomaly_score": auroc_anomaly_score
    }
    
    results_list.append(results)
    
    
    #### Save pred_probs, embeddings, and OOD mask to Numpy files
    
    # Save files here
    out_folder = f"./test_data_in_{in_dataset}_out_{out_dataset}/"
    
    # Create folder if it doesn't exist
    os.makedirs(out_folder, exist_ok=True)
    
    np.save(out_folder + "pred_probs.npy", pred_probs)
    np.save(out_folder + "embeddings.npy", features) # features = embeddings extracted from model trained on training dataset
    np.save(out_folder + "ood_mask.npy", labels) # labels in this context is whether the datapoint is OOD! It's a binary indicator, True = OOD
    
    np.save(out_folder + "in_distribution_test_dataset_class_labels.npy", in_test_dataset_class_labels) # class labels for only the in-distribution test dataset

-----------------------------------------------------
(in-distribution, out-of-distribution) dataset pair:  cifar-10 cifar-100
  Loading model...
  Generating predicted probabilities...
  Extracting learned embeddings...
  Running nearest neighbors search...
Building nearest neighbors index
  Generating scores...
-----------------------------------------------------
(in-distribution, out-of-distribution) dataset pair:  cifar-100 cifar-10
  Loading model...
  Generating predicted probabilities...


/usr/local/lib/python3.8/dist-packages/autogluon/vision/predictor/predictor.py:548: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_pred_proba[list(self._label_cleaner.cat_mappings_dependent_var.values())] = y_pred_proba['image_proba'].to_list()


  Extracting learned embeddings...


/usr/local/lib/python3.8/dist-packages/autogluon/vision/predictor/predictor.py:548: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_pred_proba[list(self._label_cleaner.cat_mappings_dependent_var.values())] = y_pred_proba['image_proba'].to_list()


  Running nearest neighbors search...
Building nearest neighbors index
  Generating scores...
-----------------------------------------------------
(in-distribution, out-of-distribution) dataset pair:  mnist roman-numeral
  Loading model...
  Generating predicted probabilities...
  Extracting learned embeddings...
  Running nearest neighbors search...
Building nearest neighbors index
  Generating scores...
-----------------------------------------------------
(in-distribution, out-of-distribution) dataset pair:  roman-numeral mnist
  Loading model...
  Generating predicted probabilities...
  Extracting learned embeddings...
  Running nearest neighbors search...
Building nearest neighbors index
  Generating scores...
-----------------------------------------------------
(in-distribution, out-of-distribution) dataset pair:  mnist fashion-mnist
  Loading model...
  Generating predicted probabilities...
  Extracting learned embeddings...
  Running nearest neighbors search...
Building neare

## Put results to a DataFrame

In [31]:
df_results = pd.DataFrame(results_list)

In [32]:
cols = [
    "in_distribution",
    "out_of_distribution",
    "auroc_one_minus_max_pred_prob",
    "auroc_entropy",
    "auroc_knn_k10_entropy",
    "auroc_entropy_knn_k10_pred_probs",
    "auroc_knn_k10_dist",
    "auroc_anomaly_score"
]

df_results[cols]

,in_distribution,out_of_distribution,auroc_one_minus_max_pred_prob,auroc_entropy,auroc_knn_k10_entropy,auroc_entropy_knn_k10_pred_probs,auroc_knn_k10_dist,auroc_anomaly_score
0,cifar-10,cifar-100,0.968904,0.968968,0.976455,0.977570,0.882912,0.890379
1,cifar-100,cifar-10,0.946950,0.955817,0.984437,0.981527,0.602993,0.551464
2,mnist,roman-numeral,0.992383,0.994569,0.998846,0.998381,0.967181,0.985315
3,roman-numeral,mnist,0.908997,0.929902,0.979364,0.930802,0.043762,0.001178
4,mnist,fashion-mnist,0.994458,0.995607,0.999404,0.999130,0.940788,0.929723
5,fashion-mnist,mnist,0.913697,0.937726,0.973560,0.959109,0.583062,0.527507


In [33]:
cols = [
    "in_distribution",
    "out_of_distribution",
    "auroc_knn_k5_entropy",
    "auroc_knn_k10_entropy",
    "auroc_knn_k15_entropy",
    "auroc_knn_k100_entropy",
]

df_results[cols]

,in_distribution,out_of_distribution,auroc_knn_k5_entropy,auroc_knn_k10_entropy,auroc_knn_k15_entropy,auroc_knn_k100_entropy
0,cifar-10,cifar-100,0.975999,0.976455,0.975870,0.971511
1,cifar-100,cifar-10,0.982368,0.984437,0.985209,0.985393
2,mnist,roman-numeral,0.998231,0.998846,0.998859,0.997038
3,roman-numeral,mnist,0.971430,0.979364,0.983795,0.993706
4,mnist,fashion-mnist,0.999309,0.999404,0.999393,0.998886
5,fashion-mnist,mnist,0.967805,0.973560,0.976127,0.986394


In [34]:
cols = [
    "in_distribution",
    "out_of_distribution",
    "auroc_knn_k1_dist",
    "auroc_knn_k5_dist",
    "auroc_knn_k10_dist",
    "auroc_knn_k15_dist",
    "auroc_knn_k100_dist",
]

df_results[cols]

,in_distribution,out_of_distribution,auroc_knn_k1_dist,auroc_knn_k5_dist,auroc_knn_k10_dist,auroc_knn_k15_dist,auroc_knn_k100_dist
0,cifar-10,cifar-100,0.844101,0.870990,0.882912,0.890921,0.952323
1,cifar-100,cifar-10,0.610539,0.608929,0.602993,0.597227,0.466928
2,mnist,roman-numeral,0.961315,0.965429,0.967181,0.968263,0.975588
3,roman-numeral,mnist,0.073345,0.051868,0.043762,0.039027,0.013868
4,mnist,fashion-mnist,0.934976,0.939818,0.940788,0.941378,0.945835
5,fashion-mnist,mnist,0.568013,0.577871,0.583062,0.586229,0.598909


## Misc

## Load pred_probs, OOD mask, and class labels for in-distribution dataset

In [105]:
in_dataset = "cifar-10"
out_dataset = "cifar-100"

out_folder = f"./test_data_in_{in_dataset}_out_{out_dataset}/"

In [106]:
pred_probs = np.load(out_folder + "pred_probs.npy")
ood_mask = np.load(out_folder + "ood_mask.npy")

# NOTE: this contains only the class labels for the in-distribution test dataset!
in_distribution_test_dataset_class_labels = np.load(out_folder + "in_distribution_test_dataset_class_labels.npy")

In [107]:
pred_probs.shape

(20000, 10)

In [108]:
ood_mask.shape

(20000,)

In [109]:
in_distribution_test_dataset_class_labels.shape

(10000,)

In [113]:


(pred_probs[:10000].argmax(axis=1) == in_distribution_test_dataset_class_labels).mean()

0.9887